In [ ]:
import json

with open('data/chatbot_respostas.json', encoding="utf8") as f:
    json_respostas = json.load(f)
    
    

In [ ]:
with open('data/chatbot_respostas.jsonl', 'w', encoding="utf8") as f:
    for entrada in json_respostas:
        resposta = {
            'resposta': entrada['resposta'],
            'categoria': entrada['categoria'],
            'fonte': 'AsimoBot'
        }
        entrada_jsonl = {
            'messages': [
                {'role': 'user', 'content': entrada['pergunta']},
                {'role': 'assistant', 'content': json.dumps(resposta, ensure_ascii=False, indent=2)}
            ]
        }
        json.dump(entrada_jsonl, f, ensure_ascii=False)
        f.write('\n')

In [ ]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

In [ ]:
file = client.files.create(
    file=open('arquivos/chatbot_respostas.jsonl',  'rb'),
    purpose='fine-tune'
)

In [ ]:
client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo"
)

In [ ]:
client.fine_tuning.jobs.list()

In [ ]:
mensagens = [{'role': 'user', 'content': 'O que é uma equação quadrática?'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

## Comparison
### Using always the same prompt
We end up using too many tokens as we have to pass in the same system prompt every

In [ ]:
system_mes = '''
Responda as perguntas em um parágrafo de até 20 palavras. Categorize as respostas no seguintes conteúdos: física, matemática, língua portuguesa ou outros.
Retorne a resposta em um formato json, com as keys: 
fonte: valor deve ser sempre AsimoBot
resposta: a resposta para a pergunta
categoria: a categoria da pergunta
'''

mensagens = [
    {'role': 'system', 'content': system_mes},
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
    ]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

resposta.usage

### Using our trained
Token usage is way lower

In [9]:
mensagens = [
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
    ]

resposta = client.chat.completions.create(
    messages=mensagens,
    model="ft:gpt-3.5-turbo-0125:personal::A8RG1Bd2",
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)
print(resposta.usage)


{
  "answer": "Uma equação quadrática é uma equação polinomial de segundo grau, geralmente na forma ax² + bx + c = 0.",
  "category": "Matemática",
  "source": "AsimoBot"
}
CompletionUsage(completion_tokens=57, prompt_tokens=16, total_tokens=73, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0))
